In [3]:
import yaml
from yaml.loader import SafeLoader
x
import numpy as np
import pandas as pd
from sklearn.base import clone
from dev_interaction_util import generate_synthetic_dev_outcomes, generate_synthetic_dev_data, set_up_interactions
from dev_interaction_util import do_scoring_loop, get_best_model, summarize_overall_df_results, do_final_fit, present_model_results, present_results_vs_ground_truth_cors
from dev_interaction_util import load_and_preprocess_data, impute_data
from ml_util import *

# Imputing with MICE
from devcvanalysis import DevAim3NestedCVManager, optimize_hypers

In [4]:


print(gethostname())
# Open the file and load the file
with open('config.yml') as f:
    all_yaml = yaml.load(f, Loader=SafeLoader)
    if gethostname() in all_yaml.keys():
        config = all_yaml[gethostname()]
    else:
        config = all_yaml['default']
        
print(config)



Benjamins-MacBook-Pro-2.local
{'dropbox_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/'}


This notebook is derived from `test_feature_selection.ipynb`.

In [5]:
dropbox_data_dir = config['dropbox_data_dir']


In [6]:
aim_3_analysis = DevAim3NestedCV()
aim_3_analysis.load_and_preprocess_data(dropbox_data_dir)

In [7]:
aim_3_analysis.load_and_preprocess_data(analysis_data)

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis3_10/lib/python3.10/site-packages/sklearn/impute/_iterative.py:713: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


# Improving fit with manual theory-driven feature

My past analysis showed that by manually removing some features before the analysis starts, we can improve performance beyond the chance performance otherwise seen.

So, it might be useful to understand how much we can improve our performance by manual feature selection before the automatic feature selection applies.

This was previously done in `test_limited_predictors.ipynb`. We tested as few as 2 distractor features. In that test, predictor features generally had correlations in the range of |r|=0.06 to 0.53, with most around 0.4 (we should confirm that because it seems fishy that PCS was detegted as an effect, but didn't model as a large predictor). With most `|r|=0.4`, this seems unrealistically high to expect, and we should aim to build a pipeline capable of detecting more subtle effects than that. An approximate `|r|=0.3` can be achieved by mixing in a predictor scaled to 8% of normal scale.

I can imagine it is plausible to cut down to as few as two self-report, one behavioral, and one neural measure per intervention, plus sex and age. That would yield 10 different variables. At the other end, we might want 10 self-report, two behavioral, and five neural measures per intervention tested, plus 6 different demographic variables--a total of 40 variables. Let's see how these would perform, as well as mid-range of 20 predictor variables. In each case we'll restrict to three valid predictors per intervention.

In [9]:
#create an empty data frame with numeric two columns, n_features and overall_score
overall_scores = pd.DataFrame(columns=['n_features','overall_score'])

In [10]:
analysis_data_imputed.columns

Index(['BSCS', 'EDM', 'BIS_11', 'PCS', 'RS', 'TRSQ',
       'ACES_neglectful_parenting', 'ACES_abuse', 'ACES_sum',
       'ACES_divorced_separated', 'ACES_household_dysfunction',
       'BFI_agreeableness', 'BFI_conscientiousness', 'BFI_extraversion',
       'BFI_neuroticism', 'BFI_openness', 'DEMO_mcarthur_social_standing',
       'IMI_effort_importance', 'IMI_value_usefulness',
       'IMI_interest_enjoyment', 'IMI_perceived_choice',
       'IMI_perceived_competence', 'NCS_get_job_done', 'NCS_intellectual_task',
       'NCS_deliberating_issues', 'NCS_like_responsibility',
       'NCS_thinking_not_exciting', 'NCS_avoid_depth', 'NCS_thinking_not_fun',
       'NCS_thought_appealing', 'NCS_think_minimally', 'NCS_prefer_complex',
       'NCS_prefer_little_thought', 'NCS_relief_not_satisfaction',
       'NCS_tasks_little_thought', 'NCS_new_solutions_to_problems',
       'NCS_abstract_thinking', 'NCS_total', 'NCS_small_daily_projects',
       'NCS_solve_puzzles', 'NCS_satisfaction_in_delibe

## 10 variables

In [12]:
#run the analysis with a limited number of predictors

overall_score = aim_3_analysis.run_full_limited_predictor_analysis(10, hyperparameter_optimizer = devcvanalysis.optimize_hypers)

#overall_scores = overall_scores.append({'n_features':10,'overall_score':overall_score},ignore_index=True)

['ni' 'san']
[1.28335298 0.42953651]
['san' 'san' 'ni' 'ichi' 'san' 'san' 'ichi' 'san' 'san' 'san' 'ni' 'ichi'
 'ichi' 'ichi' 'ichi' 'san' 'san' 'san' 'ichi' 'ichi' 'san' 'san' 'ni'
 'ni' 'ni' 'ni' 'ni' 'ni' 'ni' 'san' 'ni' 'san' 'ni' 'ichi' 'ni' 'san'
 'ni' 'ichi' 'san' 'ni' 'ni' 'ichi' 'ichi' 'ichi' 'san' 'san' 'ni' 'ni'
 'san' 'ichi' 'ichi' 'ni' 'san' 'ni' 'ichi' 'ni' 'ni' 'ni' 'ichi' 'san'
 'ni' 'ni' 'ichi' 'ni' 'ichi' 'san' 'ni' 'ni' 'ni' 'san' 'ichi' 'ni' 'san'
 'ichi' 'san' 'ni' 'san' 'ni' 'ichi' 'ichi' 'san' 'ichi' 'san' 'san' 'ni'
 'ichi' 'ni' 'ichi' 'san' 'ni' 'san' 'ni' 'ichi' 'san' 'san' 'san' 'ichi'
 'ni' 'san' 'ichi' 'ichi' 'san' 'ni' 'ichi' 'san' 'ni' 'ni' 'san' 'ni'
 'ichi' 'ni' 'ichi' 'ichi' 'ni' 'ichi' 'ichi' 'ichi' 'san' 'san' 'ichi'
 'ni' 'ni' 'ichi' 'ni' 'ni' 'ichi' 'ichi' 'san' 'san' 'ni' 'ichi' 'ni'
 'ichi' 'ichi' 'san' 'ichi' 'ni' 'san' 'san' 'ni' 'ni' 'san' 'san' 'san'
 'ichi' 'san' 'ni' 'san' 'ichi' 'ichi' 'ichi' 'ni' 'san' 'ni' 'ni' 'ni'
 'ichi' 'ni' 'ichi' '

doing permutation test on importance; this may take time.
Number of selected features: 11


,predictor,coef,feature_importance,fa_abs
12,BSCS*ni,0.972764,0.119505,0.119505
26,RS*san,0.581897,0.049022,0.049022
2,BIS_11,-0.392702,0.023960,0.023960
6,ACES_neglectful_parenting,-0.275182,0.015961,0.015961
30,ACES_sum*san,-0.156225,0.005204,0.005204
19,ACES_abuse*ni,-0.109025,0.003821,0.003821
7,ACES_abuse,0.159900,0.003787,0.003787
1,EDM,0.119865,0.003041,0.003041
5,TRSQ,0.107321,0.003011,0.003011
20,ACES_sum*ni,-0.018454,0.000429,0.000429


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:349: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  results_vs_cors = final_results_wide.merge(group_correlations, left_index=True, right_index=True, how='outer')


,"(coef, base)","(coef, ni)","(coef, san)","(feature_importance, base)","(feature_importance, ni)","(feature_importance, san)",ichi_cor,ni_cor,san_cor,abs_effect_sum
BSCS,-0.000,0.973,-0.000,0.000,0.120,0.000,-0.137,0.350,0.008,0.120
RS,NaN,NaN,0.582,NaN,NaN,0.049,0.039,-0.154,0.260,0.049
BIS_11,-0.393,-0.000,0.000,0.024,0.000,0.000,0.047,-0.383,-0.043,0.024
ACES_neglectful_parenting,-0.275,0.022,NaN,0.016,0.000,NaN,-0.046,-0.017,-0.218,0.016
ACES_abuse,0.160,-0.109,-0.000,0.004,0.004,0.000,NaN,NaN,NaN,0.008
ACES_sum,0.000,-0.018,-0.156,0.000,0.000,0.005,NaN,NaN,NaN,0.006
EDM,0.120,NaN,-0.000,0.003,NaN,0.000,0.053,0.233,-0.056,0.003
TRSQ,0.107,-0.000,0.000,0.003,0.000,0.000,0.091,-0.222,0.264,0.003


## Iterate through

In [13]:
overall_scores = pd.DataFrame(columns=['n_features','effect_size', 'overall_score'])
for predictors in [10,20,40]:
    for es in [0.06, 0.08, 0.1, 0.15, 0.2]:
        #run the analysis with a limited number of predictors
        overall_score = run_full_limited_predictor_analysis(predictors, outcome_measures, analysis_data_imputed, effect_size=es)
        overall_scores = overall_scores.append({'n_features':predictors,'effect_size':es, 'overall_score':overall_score},ignore_index=True)

['ni' 'san']
[1.28335298 0.42953651]
['san' 'san' 'ni' 'ichi' 'san' 'san' 'ichi' 'san' 'san' 'san' 'ni' 'ichi'
 'ichi' 'ichi' 'ichi' 'san' 'san' 'san' 'ichi' 'ichi' 'san' 'san' 'ni'
 'ni' 'ni' 'ni' 'ni' 'ni' 'ni' 'san' 'ni' 'san' 'ni' 'ichi' 'ni' 'san'
 'ni' 'ichi' 'san' 'ni' 'ni' 'ichi' 'ichi' 'ichi' 'san' 'san' 'ni' 'ni'
 'san' 'ichi' 'ichi' 'ni' 'san' 'ni' 'ichi' 'ni' 'ni' 'ni' 'ichi' 'san'
 'ni' 'ni' 'ichi' 'ni' 'ichi' 'san' 'ni' 'ni' 'ni' 'san' 'ichi' 'ni' 'san'
 'ichi' 'san' 'ni' 'san' 'ni' 'ichi' 'ichi' 'san' 'ichi' 'san' 'san' 'ni'
 'ichi' 'ni' 'ichi' 'san' 'ni' 'san' 'ni' 'ichi' 'san' 'san' 'san' 'ichi'
 'ni' 'san' 'ichi' 'ichi' 'san' 'ni' 'ichi' 'san' 'ni' 'ni' 'san' 'ni'
 'ichi' 'ni' 'ichi' 'ichi' 'ni' 'ichi' 'ichi' 'ichi' 'san' 'san' 'ichi'
 'ni' 'ni' 'ichi' 'ni' 'ni' 'ichi' 'ichi' 'san' 'san' 'ni' 'ichi' 'ni'
 'ichi' 'ichi' 'san' 'ichi' 'ni' 'san' 'san' 'ni' 'ni' 'san' 'san' 'san'
 'ichi' 'san' 'ni' 'san' 'ichi' 'ichi' 'ichi' 'ni' 'san' 'ni' 'ni' 'ni'
 'ichi' 'ni' 'ichi' '

doing permutation test on importance; this may take time.
Number of selected features: 3


,predictor,coef,feature_importance,fa_abs
12,BSCS*ni,0.647131,0.065302,0.065302
26,RS*san,0.170430,0.010150,0.010150
6,ACES_neglectful_parenting,-0.095414,0.003308,0.003308
10,ni,0.000000,0.000000,0.000000
11,san,0.000000,0.000000,0.000000
14,BIS_11*ni,-0.000000,0.000000,0.000000
17,TRSQ*ni,0.000000,0.000000,0.000000
18,ACES_neglectful_parenting*ni,0.000000,0.000000,0.000000
20,ACES_sum*ni,-0.000000,0.000000,0.000000
27,TRSQ*san,0.000000,0.000000,0.000000


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:349: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  results_vs_cors = final_results_wide.merge(group_correlations, left_index=True, right_index=True, how='outer')


,"(coef, base)","(coef, ni)","(coef, san)","(feature_importance, base)","(feature_importance, ni)","(feature_importance, san)",ichi_cor,ni_cor,san_cor,abs_effect_sum
BSCS,NaN,0.647,NaN,NaN,0.065,NaN,-0.137,0.277,0.027,0.065
RS,NaN,NaN,0.17,NaN,NaN,0.01,0.039,-0.128,0.211,0.010
ACES_neglectful_parenting,-0.095,0.000,NaN,0.003,0.000,NaN,-0.046,-0.021,-0.170,0.003


...

doing permutation test on importance; this may take time.
Number of selected features: 12


,predictor,coef,feature_importance,fa_abs
42,BSCS*ni,5.540987,2.517312,2.517312
44,BIS_11*ni,-4.457666,1.640721,1.640721
86,RS*san,2.321439,0.455755,0.455755
88,ACES_neglectful_parenting*san,-1.340184,0.162157,0.162157
82,BSCS*san,-0.492771,0.024445,0.024445
91,ACES_divorced_separated*san,-0.421082,0.018296,0.018296
52,ACES_household_dysfunction*ni,-0.322238,0.011071,0.011071
93,BFI_agreeableness*san,-0.303110,0.010067,0.010067
7,ACES_abuse,0.212698,0.004146,0.004146
48,ACES_neglectful_parenting*ni,0.108653,0.001537,0.001537


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:349: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  results_vs_cors = final_results_wide.merge(group_correlations, left_index=True, right_index=True, how='outer')


,"(coef, base)","(coef, ni)","(coef, san)","(feature_importance, base)","(feature_importance, ni)","(feature_importance, san)",ichi_cor,ni_cor,san_cor,abs_effect_sum
BSCS,NaN,5.541,-0.493,NaN,2.517,0.024,-0.137,0.632,-0.083,2.542
BIS_11,NaN,-4.458,NaN,NaN,1.641,NaN,0.047,-0.624,0.011,1.641
RS,NaN,NaN,2.321,NaN,NaN,0.456,0.039,-0.254,0.464,0.456
ACES_neglectful_parenting,-0.000,0.109,-1.340,0.000,0.002,0.162,-0.046,0.000,-0.423,0.164
ACES_divorced_separated,0.000,0.000,-0.421,0.000,0.000,0.018,NaN,NaN,NaN,0.018
ACES_household_dysfunction,0.000,-0.322,-0.003,0.000,0.011,0.000,NaN,NaN,NaN,0.011
BFI_agreeableness,NaN,NaN,-0.303,NaN,NaN,0.010,NaN,NaN,NaN,0.010
ACES_abuse,0.213,-0.019,-0.000,0.004,0.000,0.000,NaN,NaN,NaN,0.004


/var/folders/cj/4mb6t1f906j397tj71pxfxz00000gn/T/ipykernel_27475/3204600133.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  overall_scores = overall_scores.append({'n_features':predictors,'effect_size':es, 'overall_score':overall_score},ignore_index=True)


# Conclusion so far

The feature selection applied here hasn't helped very much. That surprises me because in test_limited_predictors, I got clear evidence that cutting down irrelevant predictors improved model performance.

One reason might be that we've actually cut down on useful predictors--unlike in `test_limited_predictors.ipynb`, we can't cheat by removing predictors we know to be irrelevant. That means we're left with less information in the model itself.

We've only really tried SelectKBest(); there might be other feature selection mechanisms that could do the job. But I don't know yet.